In [ ]:
pip install mlconfound

In [ ]:
pip install openpyxl

In [ ]:
pip install scipy

In [ ]:
from mlconfound.stats import partial_confound_test
from mlconfound.simulate import simulate_y_c_yhat
from mlconfound.plot import plot_null_dist, plot_graph

import pandas as pd
import seaborn as sns

In [ ]:
# specify home directory
def get_home_directory_with_expanduser():
    return os.path.expanduser("~")

# read in demographics of the subjects
demo = pd.read_csv("~/sciebo/Features_age/data/demo_regular.csv", sep=';')
print(demo)

In [ ]:
# read in PLS-results for the conventional features
from scipy.io import loadmat

res = loadmat("/Users/Christina/sciebo/Features_age/data/pls_av_full_10k_5c_hctsa_r50_stratf.mat")

In [ ]:
print(res)

In [ ]:
y = demo['real_age'].to_numpy()
#print(y)
sex = demo['sex'].to_numpy()
eTIV = demo['eTIV'].to_numpy()
#print(eTIV)

In [ ]:
# Check a couple of variables
type(res)
# type(demo)
# print(demo.real_age)
res.keys() # this will show you the var name of your data

In [ ]:
data = res["av_yhat"]
# print(data)

In [ ]:
num_columns = len(data[1])
print(num_columns)
#column = data[:, 0]
#print(column)

In [ ]:
# loop over features and run partial confound test for variable of interest and store results
# decide on the number of permutations!
import numpy as np
result = []
graphs = []
distr = []

# change variable of interest accordingly
for i in range(num_columns):
    y_hat = data[:, i]
    
    # The random seed is set for reproducible results. The flag return_null_dist is set so that the full permutation-based null distribution is returned, e.g. for plotting purposes.
   # ret=partial_confound_test(y, y_hat, sex, return_null_dist=True,num_perms=1000,
    #              random_state=42)
    
    ret=partial_confound_test(y, y_hat, eTIV, return_null_dist=True,num_perms=1000,
                  random_state=42)

    #p = plot_null_dist(ret)
    #pp = plot_graph(ret, y_name='age', yhat_name='predicted age', c_name='sex', outfile_base='example')
    
    result.append(ret)
    #graphs.append(pp)
    #distr.append(p)

In [ ]:
# extract only main parameters and save as dataframe
type(result)

# Create an empty list to store the data for each row
rows = []

# Iterate over the list
for i in range(0, len(result)):
    r2_y_c = result[i][0]
    r2_y_yhat = result[i][1]
    r2_yhat_c = result[i][2]
    expected_r2_yhat_c = result[i][3]
    p_ci_low = result[i][5][0]
    p_ci_high = result[i][5][1]
    p = result[i][4]
    
    row = [r2_y_c, r2_y_yhat, r2_yhat_c,expected_r2_yhat_c, p_ci_low, p_ci_high, p]
    rows.append(row)
    
# Create the DataFrame
df = pd.DataFrame(rows, columns=['r2_y_c', 'r2_y_yhat', 'r2_yhat_c', 'expected_r2_yhat_c', 'p_ci_low', 'p_ci_high', 'p'])

# Set values equal to 0 to NaN
df['p'] = df['p'].replace(0, np.nan)
print(df)

In [ ]:
# now perform fdr corrections on the p-values
from statsmodels.stats.multitest import multipletests

# Filter out NaN p-values
filtered_pvalues = df['p'].dropna()

# Perform FDR correction on the filtered p-values
rejected, corrected_pvalues, _, _ = multipletests(filtered_pvalues, method='fdr_bh')

# Create a new DataFrame with corrected p-values
df_corrected = pd.DataFrame({'p_fdr': corrected_pvalues}, index=filtered_pvalues.index)

# Merge the corrected p-values back to the original DataFrame
df['p_fdr'] = np.nan  # Initialize with NaN
df.loc[~df['p'].isna(), 'p_fdr'] = df_corrected['p_fdr']

# Print the DataFrame with corrected p-values
print(df)

In [ ]:
df

In [ ]:
# Save DataFrame to an Excel file
df.to_excel("~/sciebo/Features_age/results_all_subjects/confound_strf/confound_eTIV_hctsa2_r50_stratf.xlsx", index=False)
#df.to_excel("~/sciebo/Features_age/results_all_subjects/confound_strf/confound_sex_hctsa2_r50_stratf.xlsx", index=False)

In [ ]:
##### let's plot some results
##### try with AC=11, which is index = 95 (remember that Python starts counting with 0, not one)
##### try with center of gravity, which is index = 5980

i = 5980
# loop over features and run partial confound test for sex and store results
# decide on the number of permutations!
result = []
graphs = []
distr = []

#for i in range(num_columns):
y_hat = data[:, i]
    
# The random seed is set for reproducible results. The flag return_null_dist is set so that the full permutation-based null distribution is returned, e.g. for plotting purposes.
ret=partial_confound_test(y, y_hat, eTIV, return_null_dist=True,num_perms=1000,random_state=42)

p = plot_null_dist(ret)
pp = plot_graph(ret, y_name='age', yhat_name='predicted age', c_name='eTIV', outfile_base='example')
    
    #result.append(ret)
    #graphs.append(pp)
    #distr.append(p)